In [ ]:
pip install apache-beam[gcp]

In [ ]:
from apache_beam.options.pipeline_options import PipelineOptions

In [ ]:
RUNNER = 'DataflowRunner'
PROJECT = 'keen-enigma-342516'
JOB = 'quijote-counts-job-name'
TEMP_LOCATION = 'gs://dataflow_databits/temp'
REGION = 'us-central1'

In [ ]:
beam_options = PipelineOptions(
    runner=RUNNER,
    project=PROJECT,
    job_name=JOB,
    temp_location=TEMP_LOCATION,
    region=REGION,
)

In [ ]:
INPUT_FILE = 'gs://dataflow_databits/el_quijote.txt'
OUTPUT_FILE = 'gs://dataflow_databits/el_quijote_counts'

In [ ]:
with beam.Pipeline(options=beam_options) as pipeline:
    (
        pipeline
        | 'Read lines' >> beam.io.ReadFromText(INPUT_FILE)
        | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
        | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
        | 'Group and sum' >> beam.CombinePerKey(sum)
        | 'Format results' >> beam.Map(lambda word_count: str(word_count))
        | 'Write results' >> beam.io.WriteToText(OUTPUT_FILE)
    )